In [21]:
import boto3
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir
import streamlit as st
import base64

In [22]:
def autoplay_audio(file_path: str):
    with open(file_path, "rb") as f:
        data = f.read()
        b64 = base64.b64encode(data).decode()
        md = f"""
            <audio controls autoplay="true">
            <source src="data:audio/mp3;base64,{b64}" type="audio/mp3">
            </audio>
            """
        st.markdown(
            md,
            unsafe_allow_html=True,
        )

In [16]:
Session = boto3.Session(
        aws_access_key_id = st.secrets['aws_access_key_id'],
        aws_secret_access_key = st.secrets['aws_secret_access_key'],
        region_name = "us-east-1"
    )

In [17]:
Polly = Session.client("polly")

response = Polly.synthesize_speech(
        Text= "hello, my name is Joanna",
        OutputFormat="mp3",
        VoiceId="Joanna")

In [18]:
if "AudioStream" in response:
    with closing(response["AudioStream"]) as stream:
        output = os.path.join(gettempdir(), "speech.mp3")

        try:
            with open(output, "wb") as file:
                file.write(stream.read())
        except IOError as error:
            print("I/O error: {}".format(error))
            sys.exit(-1)

else:
    print("Could not stream audio")
    sys.exit(-1)

In [25]:
st.write("# Auto-playing Audio!")

In [27]:
autoplay_audio(output)